# Predicing user Ad-click using Logistic Regresion

#### Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

read from the file

In [2]:
df = pd.read_csv('data/advertising.csv')

In [3]:
plt.style.use('ggplot')